This script prepares for an experiment testing a set of samples. 

It creates a notebook with an entry
for each sample assay. 

A ListOfMaterials is attached to each notebook entry linking to the information and physical location of the sample.

This sort of script could be use to prepare notebooks for the scientists performing the experiement, cutting down on tedious manual data entry. 

Set up CLI and assert it's working OK. You'll need RSpace 1.70 or later, and rspace_client 2.0.0 alpha.

In [12]:
import rspace_client.inv.inv as inv
import rspace_client.eln.eln as eln
import os
inv_cli = inv.InventoryClient(os.getenv("RSPACE_URL"), os.getenv("RSPACE_API_KEY"))
eln_cli  = eln.ELNClient(os.getenv("RSPACE_URL"), os.getenv("RSPACE_API_KEY"))
inv_cli.get_workbenches()
eln_cli.get_status()

{'message': 'OK', 'rspaceVersion': '1.70.4-SNAPSHOT-2021-09-30T22:15:08Z'}

Read in sample information from a file. In these case these are Uniprot identifiers but could be anything - perhaps identifiers generated automatically as part of some automted synthesis or sample preparation

In [13]:

with open("proteins.csv", "r") as reader:
    samples = reader.read().splitlines()[1:]
print(samples[0:10])
len(samples)

['P12235', 'P12235', 'O00555', 'O00555', 'Q96F25', 'Q8TD30', 'Q01484', 'Q01484', 'Q01484', 'Q01484']


95

Now we'll create a 96 well plate to represent the location of the samples to assay

In [14]:
plate_96 = inv_cli.create_grid_container(column_count=8, row_count=12, name="LossOFFunctionMutationsAssay#1")

Now we'll create the samples

In [15]:
created_samples = []
for sample in samples[0:16]:
    print(f"Creating sample {sample}")
    created_samples.append(inv_cli.create_sample(sample, tags="loss-of-function", subsample_count=5))

Creating sample P12235
Creating sample P12235
Creating sample O00555
Creating sample O00555
Creating sample Q96F25
Creating sample Q8TD30
Creating sample Q01484
Creating sample Q01484
Creating sample Q01484
Creating sample Q01484
Creating sample P41181
Creating sample Q6ZSZ5
Creating sample P50993
Creating sample P50993
Creating sample P30049
Creating sample P30049


In [16]:
aliquots = [s['subSamples'][0] for s in created_samples ]

Here we are auto-placing items row by row, but we can also set item locations individually or column by column.

In [17]:
by_row = inv.ByRow(1,1,8,12, *subsamples)
inv_cli.add_items_to_grid_container(target_container_id=plate_96['id'], grid_placement=by_row)

We've finished adding information to Inventory. 
Now we can create a notebook with entries - 1 per sample.

Each entry has a ListOfMaterials linking to the sample.

In [18]:
nb = eln_cli.create_folder(name="Sample Assay#1", notebook=True)

## we'd get this form ID from the UI. Or we could just create unstructured documents with single text fields.
EXPERIMENT_FORM_ID = 3

for i in range(len(subsamples)):
    doc = eln_cli.create_document(name=f"assay{i}-{created_samples[i]['name']}", 
                                  form_id=EXPERIMENT_FORM_ID, parent_folder_id=nb['id'])
    
    ## this is the 2nd field - 'Objective'
    field_id = doc['fields'][1]['id']
    lom =  inv_cli.create_list_of_materials(field_id, "Assay target details", subsamples[i], description=" desc")
print(f"done - see prepared notebook {nb['globalId']}")

done - see prepared notebook NB732


At the end of this script we have:

- Added sample and location information to Inventory
- prepared a notebook with entries of structured data (Objective, Method, Results
- Created a  List of Materials linking to information about the sample.

All that's left to do now is perform the experiment! 